<font size=-1>Licensed under the Apache License, Version 2.0 (the \"License\");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at [https://www.apache.org/licenses/LICENSE-2.0](https://www.apache.org/licenses/LICENSE-2.0)

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.  See the License for the specific language governing permissions and limitations under the License.</font>

# Continuous training with TFX and Cloud AI Platform

This lab walks you through a TFX pipeline that uses **Cloud Dataflow** and **Cloud AI Platform Training** as execution runtimes. The pipeline implements a typical TFX workflow as depicted on the below diagram:

![TFX CAIP](../../../images/tfx-caip-1.png)




In [2]:
!ls ../../../images

add_trigger.png   clv_train.png		 kfp.png      lab_300.png
clv-timeline.png  configure_trigger.png  lab-env.png  tfx-caip-1.png
clv-timeline.svg  kfp-caip.png		 lab_200.png
